<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%EB%8B%A4%EB%B3%80%EB%9F%89%EB%B6%84%EC%84%9D_7%EC%9E%A5_%ED%8C%90%EB%B3%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#예제

##사후확률이용방법+판별모형 평가(분류표) (p.200)

In [ ]:
#MASS는 기본설치패키지, 따로 install필요 X
library(MASS)

##분류함수 계수를 구하는 프로그램

##예제
>
알콜 종류 3가지, 독립변수 6개, 케이스 수가 77개인 자료임, MASS패키지의 lda()함수를 이용해 판별분석하기

#실습

##1. 어느 시점을 기준으로 도산한 기업과 정상운영 기업을 대상으로 지난 2년 동안에 발생된 재무상황의 변동을 조사한 결과이다. 각 변수의 의미는 다음과 같은데 해당 자료로 판별분석과 그 결과를 해석하라
>
X1 : 운영자금/총 자산
X2 : 판매수익/총 자산
X3 : 세전총소득/총 자산
X4 : 시장가치자산/총 자산
X5 : 판매액/총 자산

##2. 공업도시와 상업도시별로 인구수(단위:만 명)와 대기오엽도를 조사한 자료, 판별분석과 결과 해석

##3. 어느 한 시점에서 도산한 기업과 도산하지 않은 기업들의 2년 전 재무현황 자료, 이 자료 중 앞부분을 이용하여 판별분석을 수행한 후, 이를 이용하여 나머지 기업에 대한 2년 후 기업들의 도산 여부를 분류하라
>
X1 : 총 부채에 대한 유동성 비율
X2 : 총 자산에 대한 순이익 비율
X3 : 총 부채에 대한 현 자산 비율
X4 : 순판매에 대한 현 자산 비율
1. 판별분석하기
2. 2년 후 자료를 분류하기